<a href="https://colab.research.google.com/github/hasune613/hello-world/blob/main/4_BERT_livedoor_news_IIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##日本語BERTでlivedoorニュースを相互情報量最大化(IIC)でクラスタリング

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234
torch.manual_seed(SEED_VALUE)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Livedoorニュースのファイルをダウンロード

! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2021-09-16 05:40:12--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  6.01MB/s    in 1.4s    

2021-09-16 05:40:15 (6.01 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [ ]:
import tarfile

# # 解凍
tar = tarfile.open("ldcc-20140209.tar.gz","r:gz")
tar.extractall('./data/livedoor/')
tar.close()

# # フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir('./data/livedoor/text/')]
print(files_folders)

# # カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir('./data/livedoor/text/') if os.path.isdir('./data/livedoor/text/'+name )]
print('カテゴリ数：',len(categories))
print(categories)



['it-life-hack', 'dokujo-tsushin', 'topic-news', 'smax', 'livedoor-homme', 'peachy', 'README.txt', 'kaden-channel', 'movie-enter', 'sports-watch', 'CHANGES.txt']
カテゴリ数： 9
['it-life-hack', 'dokujo-tsushin', 'topic-news', 'smax', 'livedoor-homme', 'peachy', 'kaden-channel', 'movie-enter', 'sports-watch']


In [ ]:
def extract_main_txt(file_name):
    with open(file_name) as text_file: 
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]
    


        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]# 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line : line != '',text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '','\t':'','\r':'','\u3000':''}
        ))# 改行やタブ、全角スペースを消す

        return text


In [ ]:
import glob
list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join('./data/livedoor/text', cat,'*.txt'))
    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files] 
    label = [cat] * len(body) # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)#appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label) 

print(list_text[0])
print(list_label[0])

デルは、2012年9月15日（土）に小中学生を対象とした「親子で体験!! デル パソコン組み立て教室」をデル宮崎カスタマーセンター（宮崎市）にて開催する。今回で5回目を迎えるこのイベントは、日ごろ使っているパソコンの「内部」がどのような仕組みになっているのかを親子で楽しく学べる機会として毎年好評を得ているイベントである。当日は親子でパソコンの仕組みについて学習したあと、実際にノートパソコンを組み立てる。組み立てたノートパソコンはそのまま持ち帰ることができる。今回組み立てるパソコンはデル自慢の「Inspiron 13z」と「XPS 13」だ。「Inspiron 13z」は、驚くほどのコンパクトさと高いパフォーマンスを兼ね備えたモバイルノート。また「XPS 13」は今年3月に販売されて以来、堅牢性、機能性、デザイン性などについて幅広いユーザーの支持を集めると、パソコン購入の検討をしているユーザーを対象としたアンケートにおいて「興味」「デザイン」「総合評価（プライベート利用）」の3部門でユーザー評価No.1 を獲得したデル自慢のUltrabookだ。今回のイベントは、デルのCSR（企業の社会的責任）活動の一環として行われるものだ。デルは米国法人や日本法人を含める各国において、子供たちや若者がテクノロジーに親しめるようなプログラムを世界各国で積極的に展開している。日本ではデル株式会社が「お客さまセンター」を設置している宮崎で「パソコン組み立て教室」を例年実施してきた経緯がある。最新の技術に文字通り「触れる」機会を設け、子供たちの可能性や探究心を引出すことで、テクノロジーやものづくりへの興味・関心を持ってもらうことが目的だ。こうしたイベントに参加することで将来のPC設計者が登場してくる助けになることは間違いない。また、親子のコミュニケーションを図るよい機会にもなるだろう。■「親子で体験!! デル パソコン組み立て教室」実施概要・日時：2012年9月15日（土）・受付：午後1時〜 ○パソコン教室：午後1時30分〜午後5時・会場：デル株式会社宮崎カスタマーセンター住所：宮崎県宮崎市橘通東4-8-1 カリーノ宮崎4F（日豊本線宮崎駅より徒歩10分）・講師：デル株式会社テクニカルサポート担当社員・教 材：①『InspironTM 13z』/②『XPSTM 13』・参加費用：①44,9

In [ ]:
import pandas as pd

df = pd.DataFrame({'text':list_text,'label': list_label})

df.shape


(7376, 2)

In [ ]:
import pprint

In [ ]:
# カテゴリーの辞書を作成
dec_id2cat = dict(zip(list(range(len(categories))),categories))
dec_cat2id = dict(zip(categories,list(range(len(categories)))))
pprint.pprint(dec_id2cat)
print('\n')
pprint.pprint(dec_cat2id)

{0: 'it-life-hack',
 1: 'dokujo-tsushin',
 2: 'topic-news',
 3: 'smax',
 4: 'livedoor-homme',
 5: 'peachy',
 6: 'kaden-channel',
 7: 'movie-enter',
 8: 'sports-watch'}


{'dokujo-tsushin': 1,
 'it-life-hack': 0,
 'kaden-channel': 6,
 'livedoor-homme': 4,
 'movie-enter': 7,
 'peachy': 5,
 'smax': 3,
 'sports-watch': 8,
 'topic-news': 2}


In [ ]:
# DataFrameにカテゴリーindexの列を作成
df['label_index'] = df['label'].map(dec_cat2id)
df.head()

,text,label,label_index
0,デルは、2012年9月15日（土）に小中学生を対象とした「親子で体験!! デル パソコン組み...,it-life-hack,0
1,アスクは、iPhoneやiPadを装着して目ざまし時計代わりにできる「オーディオアラームドッ...,it-life-hack,0
2,先日来日した折に日本のテレビ局の音楽番組の「ミュージックステーション」（通称：Mステ、テレビ...,it-life-hack,0
3,相変わらず精力的にITやPC、モバイルネタまで様々な話題を探しては提供してくれる「デジ通」。...,it-life-hack,0
4,今夏、都内は猛暑日が続く中、オリンピック観戦で、スポーツしたい！と思ってしまった人も多いかと...,it-life-hack,0


In [ ]:
# label列を消去し、text, indexの順番にする
df = df.loc[:,['text','label_index']]
df.head()

,text,label_index
0,デルは、2012年9月15日（土）に小中学生を対象とした「親子で体験!! デル パソコン組み...,0
1,アスクは、iPhoneやiPadを装着して目ざまし時計代わりにできる「オーディオアラームドッ...,0
2,先日来日した折に日本のテレビ局の音楽番組の「ミュージックステーション」（通称：Mステ、テレビ...,0
3,相変わらず精力的にITやPC、モバイルネタまで様々な話題を探しては提供してくれる「デジ通」。...,0
4,今夏、都内は猛暑日が続く中、オリンピック観戦で、スポーツしたい！と思ってしまった人も多いかと...,0


In [ ]:
# 順番をシャッフルする
df = df.sample(frac = 1, random_state = 123).reset_index(drop=True)
df.head()

,text,label_index
0,こんにちは、ビデオエディターを目指している小島暢華です。普段は写真についてのコラムを書いてい...,6
1,現金と同じように利用できる「楽天ポイント」が、楽天の提携先であるオンラインゲーム会社「ネクソ...,6
2,at_STARBUCKS_Wi2を試してきた！ちょうど1週間前の2日（月）からコーヒーチェー...,3
3,メールマガジン配信サービス「BLOGOS メルマガ」は「あんざいゆきのAndroidアプリU...,6
4,女子なら一度は、大勢のファンに囲まれて歌っているスターに憧れた時期があるはず。そんな女子の夢...,5


In [ ]:
#txvファイルで保存する

#全体の2割の文章数
len_02 = len(df)//5

# 前から2割をテストデータとする
df[:len_02].to_csv('./test.tsv',sep ='\t',index = False, header = None)
print(df[:len_02].shape)
# 後ろ8割を訓練&検証データとする
df[len_02:].to_csv('./train_eval.tsv',sep='\t',index=False,header=None)
print(df[len_02:].shape)


(1475, 2)
(5901, 2)


In [ ]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install transformers==2.9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

from transformers.modeling_bert import BertModel
from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

In [ ]:
! pip install mecab-python3==0.996.5

     |████████████████████████████████| 17.1 MB 72 kB/s 
  Attempting uninstall: mecab-python3
    Found existing installation: mecab-python3 1.0.4
    Uninstalling mecab-python3-1.0.4:
      Successfully uninstalled mecab-python3-1.0.4


In [ ]:
! pip install unidic-lite

     |████████████████████████████████| 47.4 MB 40 kB/s 
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658836 sha256=eef3f85d9194920f3d7d88b81aa73ad1983dfb1e1f72643588f490493014d2fb
  Stored in directory: /root/.cache/pip/wheels/de/69/b1/112140b599f2b13f609d485a99e357ba68df194d2079c5b1a2
Successfully built unidic-lite


In [ ]:
tokenizer = BertJapaneseTokenizer.from_pretrained(
    'bert-base-japanese-whole-word-masking'
)

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

In [ ]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義
max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt')[0]


TEXT = data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL =data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [ ]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = data.TabularDataset.splits(
    path='./', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])

In [ ]:
dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901)

In [ ]:
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())

4426
1475
1475


In [ ]:
batch_size = 16

dl_train = data.Iterator(dataset_train,batch_size=batch_size,train=True)
dl_eval = data.Iterator(dataset_eval,batch_size=batch_size,train=False,sort=False)
dl_test = data.Iterator(dataset_test,batch_size=batch_size,train=False,sort=False)

dataloaders_dict = {'train':dl_train,'val':dl_eval}


In [ ]:
batch = next(iter(dl_test))

print(batch)
print(batch.Text[0].shape)
print(batch.Label.shape)


[torchtext.legacy.data.batch.Batch of size 16]
	[.Text]:('[torch.LongTensor of size 16x512]', '[torch.LongTensor of size 16]')
	[.Label]:[torch.LongTensor of size 16]
torch.Size([16, 512])
torch.Size([16])


In [ ]:
from transformers.modeling_bert import BertModel

model = BertModel.from_pretrained('bert-base-japanese-whole-word-masking')
model.eval()
print('ネットワーク設定完了')

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

ネットワーク設定完了


In [ ]:
def vectorize_with_bert(net, dataloader):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    net.to(device)
    torch.backends.cudnn.benchmark =True
    batch_size = dataloader.batch_size


    for index, batch in enumerate(dataloader):
        
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        inputs = batch.Text[0].to(device)
        labels = batch.Label.to(device)

        with torch.set_grad_enabled(False):

            result = net(inputs)
            vec_0 = result[0]
            vec_0 = vec_0[:,0,:]
            vec_0 = vec_0.view(-1, 768)

            if index == 0:
                list_text = vec_0
                list_label = labels
            else:
                list_text = torch.cat([list_text,vec_0],dim=0)
                list_label = torch.cat([list_label,labels],dim=0)
    return list_text,list_label


In [ ]:
list_text_train, list_label_train = vectorize_with_bert(model,dl_train)
list_text_eval, list_label_eval = vectorize_with_bert(model,dl_eval)
list_text_test, list_label_test = vectorize_with_bert(model,dl_test)

In [ ]:
from torch.utils.data import TensorDataset

dataset_bert_train = TensorDataset(list_label_train.view(-1,1),list_text_train)
dataset_bert_eval = TensorDataset(list_label_eval.view(-1,1),list_text_eval)
dataset_bert_test = TensorDataset(list_label_test.view(-1,1),list_text_test)


In [ ]:
from torch.utils.data import DataLoader

batch_size =1024

dl_bert_train =DataLoader(dataset_bert_train,batch_size = batch_size,shuffle=True,drop_list=True)
dl_bert_eval =DataLoader(dataset_bert_eval,batch_size = batch_size,shuffle=False)
dl_bert_test =DataLoader(dataset_bert_test,batch_size = batch_size,shuffle=False)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:

OVER_CLUSTRING_RATE = 10

class NetIIC(nn.Module):
    def __init__(self):
        super(NetIIC, self).__init__()

        # multi-headは今回しない
        self.conv1 = nn.conv1d(1,400, kernel_size = 768, stride=1,padding=0)
        self.bn1 = nn.BatchNorm1d(400)
        self.conv2 = nn.conv1d(1,300, kernel_size = 400, stride=1,padding=0)
        self.bn2 = nn.BatchNorm1d(300)
        self.conv3 = nn.conv1d(1,300, kernel_size = 300, stride=1,padding=0)
        self.bn3 = nn.BatchNorm1d(300)

        self.fc1 = nn.Linear(300,250)
        self.bnfc1 = nn.BatchNorm1d(250)

        # livedoorニュースの9カテゴリに対応するかな？と期待する9分類
        self.fc2 = nn.Linear(250,9)

        # overclustering
        # 実際の想定よりも多めにクラスタリングさせることで、
        # ネットワークで微細な変化を捉えられるようにする
        self.fc2_overclustering =nn.Linear(250, 9*OVER_CLUSTRING_RATE)

    def forward(self,x):
        x = view (x.size(0),1,-1)
        x = F.relu(self.bn1(self.conv1(x)))

        x = view (x.size(0),1,-1)
        x = F.relu(self.bn2(self.conv3(x)))

        x = view (x.size(0),1,-1)
        x = F.relu(self.bn3(self.conv3(x)))

        x = view (x.size(0),1,-1)
        x_prefinal = F.relu(self,bnfc1(self.fc1(x)))

        y = F.softmax(self.fc(x_prefinal),dim=1)
        y_overclustering = F.softmax(self.fc2_overclustering(
            x_prefinal),dim=1) # overclustering
        return y, y_overclustering


In [ ]:
import torch.nn.init as init

def weight_init(m):
    # 重み初期化
    if isinstance(m,nn.conv1d):
        init.normal_(m.weight.data)
        if m.bias is not None:
            init.normal_(m.bias.data)
    elif isinstance(m,nn.BatchNorm1d):
        init.normal_(m.weight.data, mean=1,std=0.02)
        init.constant_(m,nn.Linear)
    elif isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight.data)

        if m.bias is not None:
            init.normal_(m.bias.data)
    

In [ ]:
import sys

def compute_joint(x_out, x_tf_out:
    bn , k = x_out.size()
    assert (f'x_tf_out.size(0) == bn and x_tf_out.size(1)==k),
    {bn}{k}{x_tf_out.size(0)}{x_tf_out.size(1)}')
    p_i_j = x_out.unsqueeze(2) * x_tf_out.unsqueeze(1) # bn, k, k
    p_i_j = p_i_j.sum(dim=0)# k, k
    p_i_j = (p_i_j + p_i_j.t())/ 2. # symmetrise
    p_i_j = p_i_j / p_i_j.sum() # normalise
    return p_i_j




In [ ]:
def IID_loss (x_out,x_tf_out,EPS =sys.float_info.epsilon):

    bs,k = x_out.size()
    p_i_j = compute_joint(x_out, x_tf_out)
    assert (p_ij.size() == (k, k))

    p_i = p_i_j.sum(dem=1).view(k ,1).expand(k,k)
    p_j = p_i_j.sum(dem=1).view(1 ,k).expand(k,k)

    p_i_j = torch.shere(p_i_j <EPS, torch.tensor(
        [EPS], device = p_i_j.device),p_i_j)
    p_j = torch.shere(p_j <EPS, torch.tensor([EPS], device = p_j.device),p_j)
    p_i = torch.shere(p_i <EPS, torch.tensor([EPS], device = p_i.device),p_i)

    alpha = 2.0
    loss = (-p_i_j * (torch.log(p_i_j) -alpha * torch.log(p_j) - alpha * torch.log(p_i))).sum()

    return loss


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tensor_std = list_text_train.std(dim = 0).to(device)

def perturb_data(x):
    y = x.clone()
    noise = torch.randn(lin(tensor_std)).to(device)* tensor_std*2.0
    noise = noise.expand(x.shape[0], -1)
    y += noise

    return y

In [ ]:
def train (total_epoch ,model, train_loader , optimizer, device):
    model.train()

    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=2,T_mult=2,eta_min=0)
    
    for epoch in range(total_epoch):
        for batch_idx , (target, data) in enumerate(train_loader):
            scheduler.step()

            data_perturb = perturb_data(data)

            data = data.to(device)
            data_perturb = data_perturb.to(device)
            optimizer.zero_grad()
            output, output_overclustering = model(data)
            output_pertub, output_perturb_overclustering = model(data_perturb)

            loss1 = IID_loss(output,output_preturb)
            loss2 = IID_loss(output_overclustering,output_perturb_overclustering)
            loss = loss1 + loss2

            loss.backward()
            optimizer.step()

        if epoch % 50 ==0:
            print(f'Train Epoch {epoch}\tLoss1 : {loss1.item():.6f}\tLoss2:{loss2.item():.6f}\tLoss_total:{loss1.item()* loss2.item():.6f}')
    return model, optimizer


In [ ]:
total_epoch = 1
optimizer = torch.optim.Adam(net.parameters(), lr = 5e-4)

model_trained , optimizer = train(
    total_epoch, net, dl_bert_train, optimizer,device)


In [ ]:
import numpy as np

dl_bert_test  = DataLoader(
    dataset_bert_test, batch_size = 1, shuffle=False
)

def test (model, device, test_loader):
    model.eval()

    out_targs = []
    ref_targs = []

    total_num = len(test_loader)
    output_list = np.zeros((tortal_num, 2))

    with torch.no_grad():
        for batch_idx , (target, data)in enumerate(test_loader):
            data = data.to(davice)
            target = target.to(device)
            outputs,outputs_overclustering = model(data)

            out_targs.append(outputs.argmax(dim=1).cpu())
            ref_targs.append(target.cpu())
            
            output_list[batch_idx,0] = target.cpu()
            output_list[batch_idx,1] = outputs.argmax(dim = 1).cpu()

    out_targs = torch.cat(out_targs)
    reh_targs = torch.cat(ref_targs)

    return out_targs.view(-1,1).numpy(), ref_targs.numpy(),output_list

In [ ]:
out_targs, ref_targs, output_list = test(model_trained,device,dl_bert_test)

In [ ]:
matrix = np.zeros((9,9))

for i in range(len(out_targs)):
    row = ref_targs[i]
    col = out_targs[i]
    matrix[row][col] +=1

np.set_printoptions(suppress=True)
print(matrix)


In [ ]:
import pandas as pd

df2 = pd.DataFrame(output_list)
df2.columns=['正解クラス','推定クラス']
df2.head()

In [ ]:
df2[(df2['正解クラス'] == 0) & (df2['推定クラス']==5)].head()

In [ ]:
print(df.iloc[21,0][:300])
print(df.iloc[59,0])